In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from envs.trading_env import MyTradingEnv
from agents.classical.qlearning_agent import QLearningAgent
from agents.classical.sarsa_agent import SarsaAgent
from agents.classical.sarsa_lambda_agent import SarsaLambdaAgent
from agents.classical.monte_carlo_agent import MonteCarloAgent
from training.training_manager import TrainingManager
from training.dataclasses import TrainingConfig

def retrain_with_fixes():
    """Переобучение агентов с исправлениями"""
    
    print("="*80)
    print("ПЕРЕОБУЧЕНИЕ АГЕНТОВ С ИСПРАВЛЕНИЯМИ")
    print("="*80)
    
    # Загружаем данные
    data_path = "../data/data_1h_2023.csv"
    df = pd.read_csv(data_path, index_col=0, parse_dates=True, date_format="iso8601")
    print(f"Загружено данных: {len(df)} строк")
    
    # Создаем окружение с улучшенными настройками
    def create_env():
        return MyTradingEnv(
            df=df.copy(),
            initial_balance=1000,
            window_size=10,
            commission=0.001,  # 0.1% комиссия
            slippage=0.0005,   # 0.05% проскальзывание
            reward_scaling=0.01,  # Уменьшаем масштаб наград
            lambda_drawdown=0.3,  # Увеличиваем штраф за просадку
            lambda_hold=0.05      # Уменьшаем штраф за удержание
        )
    
    # Конфигурации для каждого агента
    configs = {
        "Q-Learning": TrainingConfig(
            agent_name="Q-Learning_v2",
            agent_type="Q-Learning",
            n_episodes=3000,
            learning_rate=0.05,
            discount_factor=0.97,
            epsilon_start=0.3,
            epsilon_end=0.01,
            epsilon_decay=0.998,
            eval_frequency=100,
            save_frequency=500
        ),
        "SARSA": TrainingConfig(
            agent_name="SARSA_v2",
            agent_type="SARSA",
            n_episodes=4000,  # Больше эпизодов для SARSA
            learning_rate=0.03,
            discount_factor=0.95,  # Меньше дисконт
            epsilon_start=0.2,
            epsilon_end=0.005,
            epsilon_decay=0.999,
            eval_frequency=100,
            save_frequency=500
        ),
        "SARSA-λ": TrainingConfig(
            agent_name="SARSA-λ_v2",
            agent_type="SARSA-λ",
            n_episodes=3500,
            learning_rate=0.02,
            discount_factor=0.96,
            epsilon_start=0.25,
            epsilon_end=0.01,
            epsilon_decay=0.997,
            lambda_param=0.8,  # Добавляем lambda
            eval_frequency=100,
            save_frequency=500
        ),
        "Monte Carlo": TrainingConfig(
            agent_name="Monte Carlo_v2",
            agent_type="Monte Carlo",
            n_episodes=2000,  # Меньше эпизодов для MC
            learning_rate=0.01,  # Меньше learning rate
            discount_factor=0.90,  # Меньше дисконт
            epsilon_start=0.1,  # Меньше случайности
            epsilon_end=0.001,
            epsilon_decay=0.995,
            eval_frequency=50,  # Чаще оценка
            save_frequency=250
        )
    }
    
    agents = {
        "Q-Learning": QLearningAgent,
        "SARSA": SarsaAgent,
        "SARSA-λ": SarsaLambdaAgent,
        "Monte Carlo": MonteCarloAgent
    }
    
    manager = TrainingManager()
    
    results = {}
    
    for name, agent_class in agents.items():
        print(f"\n{'='*60}")
        print(f"ОБУЧЕНИЕ: {name}")
        print(f"{'='*60}")
        
        try:
            # Создаем агента с улучшенными параметрами
            if name == "SARSA-λ":
                agent = agent_class(lambda_param=0.8)
            else:
                agent = agent_class()
            
            # Настраиваем параметры
            config = configs[name]
            
            # Переопределяем некоторые параметры
            agent.learning_rate = config.learning_rate
            agent.discount_factor = config.discount_factor
            agent.epsilon = config.epsilon_start
            agent.epsilon_start = config.epsilon_start
            agent.epsilon_end = config.epsilon_end
            agent.epsilon_decay = config.epsilon_decay
            
            # Создаем новое окружение для обучения
            env = create_env()
            
            # Обучаем
            result = manager.train_agent(
                agent=agent,
                env=env,
                config=config,
                experiment_name=f"exp_{name.lower().replace('-', '').replace(' ', '')}_v2",
                verbose=True
            )
            
            results[name] = result
            
            print(f"\n✅ {name} обучен успешно!")
            
        except Exception as e:
            print(f"❌ Ошибка обучения {name}: {e}")
    
    # Сравнение результатов
    print("\n" + "="*80)
    print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ ПЕРЕОБУЧЕНИЯ")
    print("="*80)
    
    comparison = []
    for name, result in results.items():
        if result:
            # Тестируем обученного агента
            env_test = create_env()
            
            # Создаем тестового агента
            agent = agent_class()
            agent.load(result['final_agent_path'])
            
            # Оценка
            state, _ = env_test.reset()
            done = False
            
            while not done:
                action = agent.select_action(state, training=False)
                next_state, reward, done, _, _ = env_test.step(action)
                state = next_state
            
            metrics = env_test.get_metrics()
            
            comparison.append({
                'Агент': name,
                'Конечный баланс': env_test._portfolio_value,
                'Доходность %': (env_test._portfolio_value / 1000 - 1) * 100,
                'Сделок': metrics.get('total_trades', 0),
                'Win Rate %': metrics.get('win_rate', 0),
                'Средний PnL': metrics.get('avg_pnl', 0)
            })
    
    if comparison:
        df_comp = pd.DataFrame(comparison)
        df_comp = df_comp.sort_values('Конечный баланс', ascending=False)
        
        print("\nРезультаты:")
        print(df_comp.to_string(index=False, float_format=lambda x: f"{x:.2f}"))
        
        # Сохраняем
        df_comp.to_csv("retraining_comparison.csv", index=False)
        print(f"\n✅ Результаты сохранены в retraining_comparison.csv")

if __name__ == "__main__":
    retrain_with_fixes()